In [1]:
# Modify the file 00_setup.py to define input/output file paths on your system
# The information in 00_setup.py will be used across notebooks
from importlib.machinery import SourceFileLoader
setup = SourceFileLoader("setup", "./00_setup.py").load_module()

# GNN - Data for Unsupervised Model

Create Stellargraph objects for unsupervised GNN runs.  The goal is to get embeddings for use in e.g. XGBoost models.

Here I use raw training features, with no label feature information

The unsupervised model has both business and NAICS nodes, and uses NAICS nodes as the head type.

*This script takes about 30 minutes on my MacBook Air*

In [2]:
import pandas as pd
import numpy as np

In [3]:
from pathlib import Path
import importlib
import pickle, re

In [4]:
import stellargraph as sg
from stellargraph import StellarGraph

2024-01-21 13:39:07.958290: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-01-21 13:39:07.958312: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-01-21 13:39:07.958319: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-01-21 13:39:07.958353: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-21 13:39:07.958370: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Import Data

In [2]:
## NN scaled input data
comb_df = pd.read_parquet(Path(setup.temp_path).joinpath('10_DATA_combined_scaled_all.parquet'))

NameError: name 'pd' is not defined

In [18]:
with open(Path(setup.temp_path).joinpath('10_DATA_features.pkl'), 'rb') as fin:
    imputed_features = pickle.load(fin)

In [21]:
num_feat =  [c for c in imputed_features if c != 'NAICS']
print(num_feat)

['NoEmp', 'CreateJob', 'LowDoc', 'DisbursementGross', 'new_business', 'urban_flag', 'franchise_flag', 'missingindicator_LowDoc', 'missingindicator_new_business', 'missingindicator_urban_flag']


## Create Edge Map

I will have edges between a business and its NAICS, as well as edges to same-sector NAICS nodes.  This section creates general edge data, which may be filtered to create graphs

##### Create NAICS map to same-sector NAICS

In [27]:
sector_map = comb_df[['NAICS', 'NAICS_sector']].drop_duplicates()

In [28]:
naics_all_sectors = sector_map.merge(sector_map.rename(columns={'NAICS':'NAICS_sim'}),
                                     how='left', on ='NAICS_sector')
naics_all_sectors = naics_all_sectors[naics_all_sectors['NAICS'] != naics_all_sectors['NAICS_sim']]

In [29]:
naics_all_sectors.sample(5)

,NAICS,NAICS_sector,NAICS_sim
28168,311812,31-33,335912
105893,332722,31-33,339993
231721,311320,31-33,333921
228905,331525,31-33,313113
131395,326111,31-33,327910


##### Functions for creating index for NAICS

In [30]:
def get_naics_index(naics_seq):
    return 'n_' + naics_seq

##### Get edges from businesses to their exact NAICS

In [31]:
# Edges from businesses to their NAICS code
naics_info_df = comb_df[['LoanNr_ChkDgt', 'NAICS', 'dset']].copy() \
    .rename(columns={'LoanNr_ChkDgt':'source'}) 
naics_info_df['target'] = get_naics_index(naics_info_df['NAICS'])

edges_business_naics = naics_info_df[['source', 'target', 'dset']].copy()
edges_business_naics['type'] = 'loan_naics'

##### Get edges from buisinesses to related NAICS codes

In [32]:
naics_info_df.head(3)

,source,NAICS,dset,target
index,,,,
795491,8521013004,421690,train,n_421690
623503,6206434009,423440,train,n_423440
142581,2048036005,722110,train,n_722110


In [33]:
edges_businesses_naics_sim = naics_info_df.drop(columns='target', errors='ignore') \
    .merge(naics_all_sectors, how='inner', on='NAICS')
edges_businesses_naics_sim['target'] = get_naics_index(edges_businesses_naics_sim['NAICS_sim'])
edges_businesses_naics_sim = edges_businesses_naics_sim[['source', 'target', 'dset']]
edges_businesses_naics_sim['type'] = 'loan_sector_naics'
print(f'Sector NAICS edges: {edges_businesses_naics_sim.shape}')

Sector NAICS edges: (67912366, 4)


In [35]:
edges_all = pd.concat([edges_business_naics, edges_businesses_naics_sim], axis=0) \
    .reset_index(drop=True)

In [36]:
edges_all['type'].value_counts()

type
loan_sector_naics    11413065
loan_naics             688081
Name: count, dtype: int64

In [37]:
print(edges_all[['source', 'target']].drop_duplicates().shape)
print(edges_all.shape)

(12101146, 2)
(12101146, 4)


In [38]:
edges_all.to_parquet(Path(setup.temp_path).joinpath('40_DATA_graph_edges.parquet'))

## Get Node Features
Use the scaled NN data.  For NAICS, just set an indicator for now.  

In [39]:
features_business = comb_df[['LoanNr_ChkDgt', 'dset'] + num_feat] \
    .set_index('LoanNr_ChkDgt')
features_business.to_parquet(Path(setup.temp_path).joinpath('40_DATA_graph_node_features_loans.parquet'))

In [40]:
# NAICS features - Just use indicator
features_naics = comb_df[['NAICS']].drop_duplicates()
features_naics['feat'] = 1
features_naics['source'] = get_naics_index(features_naics['NAICS'])
features_naics.set_index('source', inplace=True) 
features_naics.drop(columns='NAICS', inplace=True) 
print(features_naics.shape)
features_naics.to_parquet(Path(setup.temp_path).joinpath('40_DATA_graph_node_features_naics.parquet'))

(1311, 11)


## Stellargraph - train and validation
Graph without test nodes (including set aside NAICS)

In [ ]:
# Get only relevant edges and node data
edges_train_val, features_business_train_val, features_naics_train_val = \
    sg_gnn.limit_data(edges_all, features_business, features_naics,
                     ['train', 'val'])

In [ ]:
print(f'Train+val graph data')
print(f'business features, start rows {features_business.shape[0]}, end {features_business_train_val.shape[0]}')
print(f'naics features, start rows {features_naics.shape[0]}, end {features_naics_train_val.shape[0]}')

In [ ]:
sba_graph_train_val = StellarGraph({'LoanNr_ChkDgt':features_business_train_val,
                                    'NAICS': features_naics_train_val},
                                   edges_train_val, 
                                   source_column="source", target_column="target",
                                   edge_type_column="type")

In [15]:
print(sba_graph_train_val.info())

StellarGraph: Undirected multigraph
 Nodes: 634346, Edges: 11132277

 Node types:
  LoanNr_ChkDgt: [633035]
    Features: float32 vector, length 10
    Edge types: LoanNr_ChkDgt-loan_naics->NAICS, LoanNr_ChkDgt-loan_sector_naics->NAICS
  NAICS: [1311]
    Features: float32 vector, length 9
    Edge types: NAICS-loan_naics->LoanNr_ChkDgt, NAICS-loan_sector_naics->LoanNr_ChkDgt

 Edge types:
    LoanNr_ChkDgt-loan_sector_naics->NAICS: [10499242]
        Weights: all 1 (default)
        Features: none
    LoanNr_ChkDgt-loan_naics->NAICS: [633035]
        Weights: all 1 (default)
        Features: none


In [16]:
# Save stellargraph object
with open(Path(setup.temp_path).joinpath('40_DATA_stellargraph_train_val.pkl'), 'wb') as fout:
      pickle.dump(sba_graph_train_val, fout)

## Stellargraph - all nodes

In [22]:
sba_graph_all = StellarGraph({'LoanNr_ChkDgt':features_business.drop(columns=['dset']) ,
                              'NAICS': features_naics.drop(columns=['dset']},
                             edges_all.drop(columns=['dset']).drop_duplicates(),
                             source_column="source", target_column="target",
                             edge_type_column="type")

In [23]:
print(sba_graph_all.info())

StellarGraph: Undirected multigraph
 Nodes: 689392, Edges: 12101146

 Node types:
  LoanNr_ChkDgt: [688081]
    Features: float32 vector, length 10
    Edge types: LoanNr_ChkDgt-loan_naics->NAICS, LoanNr_ChkDgt-loan_sector_naics->NAICS
  NAICS: [1311]
    Features: float32 vector, length 9
    Edge types: NAICS-loan_naics->LoanNr_ChkDgt, NAICS-loan_sector_naics->LoanNr_ChkDgt

 Edge types:
    LoanNr_ChkDgt-loan_sector_naics->NAICS: [11413065]
        Weights: all 1 (default)
        Features: none
    LoanNr_ChkDgt-loan_naics->NAICS: [688081]
        Weights: all 1 (default)
        Features: none


In [24]:
# Save stellargraph object
with open(Path(setup.temp_path).joinpath('40_DATA_stellargraph_all.pkl'), 'wb') as fout:
      pickle.dump(sba_graph_all, fout)